Course Instructor: Bernd Neumayr, JKU

# UE06: SHACL Rules

Complete the **10 tasks (1 point per task)** in the `5. SHACL Rules` sheet of `SemAI.jar` first and then transfer them to this notebook.

For each task include:
- A headline including the task number
- The task description 
- The data graph and your solution (the shapes graph including the rules) in executable form
- Print out out the the data graph including derived statements (after execution of the rules). 

## Preparations

install and import rdflib and pyshacl

In [ ]:
# Install required packages in the current Jupyter kernel
!pip install -q rdflib 
!pip3 install -q pyshacl

In [ ]:
# Imports
from rdflib import Graph, Literal, RDF, URIRef, BNode, Namespace, Dataset
from rdflib.namespace import FOAF , XSD , RDFS 
from rdflib.plugins.sparql.processor import SPARQLResult
from rdflib.namespace import NamespaceManager

from pyshacl import validate

import pandas as pd

def sparql_select(graph,query,use_prefixes=True):
  results = graph.query(query)          # execute the query against the graph, resulting in a rdflib.plugins.sparql.processor.SPARQLResult
  rows = [ { var : res[var].n3(graph.namespace_manager) if (isinstance(res[var],URIRef) and use_prefixes) else res[var] for var in results.vars } for res in results ]     
                                        # construct a list of dictionaries, as intermediate format to construct the pandas DataFrame, use prefixes to abbreviate URIs                
  return pd.DataFrame(rows,columns=results.vars)        
                                        # return a pandas DataFrame constructed from the list of dictionaries, with the variables from the result set as columns      

def validation_report_as_dataframe(validation_report):
  df = sparql_select(results_graph,"""
		SELECT  ?focusNode ?resultPath ?value ?sourceConstraintComponent ?sourceShape ?resultMessage
		WHERE
  		{ ?vr	a sh:ValidationResult ;
						sh:focusNode ?focusNode ;
						sh:sourceConstraintComponent ?sourceConstraintComponent ;
						sh:sourceShape ?sourceShape ;
						sh:resultMessage ?resultMessage .					 
				OPTIONAL { ?vr sh:value ?value . }
				OPTIONAL { ?vr sh:resultPath ?resultPath . }
  		}
  """,use_prefixes=True)
  return df

def shacl_validate(dg,sg):
  return validate(dg,shacl_graph=sg,
      inference='rdfs',
      abort_on_first=False,
      allow_infos=False,
      allow_warnings=False,
      meta_shacl=False,
      advanced=False,
      js=False,
      debug=False)  
  

def shacl_validate_with_rules(dg,sg):
	return validate(dg,shacl_graph=sg,
      inference='rdfs',
      abort_on_first=False,
      allow_infos=False,
      allow_warnings=False,
      meta_shacl=False,
      advanced=True,
      iterate_rules=True, inplace=True,
      js=False,
      debug=False)


# Example


In [ ]:
# Example
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

	<Employee> rdfs:subClassOf <Person> .
	<Peter> a <Employee> ;
	   :hasParent <Mary> . 
    <Mary> a <Employee> ;
      :hasParent <Susan> ;
      :employer <Acme> .
  <Susan> a <Employee> ;
      :hasParent <John> .
""")
print('-----------Before----------')
print(dg.serialize(format='turtle'))
print('-----------After----------')

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   <http://example.org/>

	<Prefixes> sh:declare 
  [ sh:prefix "rdf";
    sh:namespace "http://www.w3.org/1999/02/22-rdf-syntax-ns#"^^xsd:anyURI ] ,
  [ sh:prefix "rdfs";
    sh:namespace "http://www.w3.org/2000/01/rdf-schema#"^^xsd:anyURI ] ,
  [ sh:prefix "sh";
    sh:namespace "http://www.w3.org/ns/shacl#"^^xsd:anyURI ] ,
  [ sh:prefix "xsd";
    sh:namespace "http://www.w3.org/2001/XMLSchema#"^^xsd:anyURI ] ,
  [ sh:prefix "";
    sh:namespace "http://example.org/"^^xsd:anyURI ] .

	<Person> a rdfs:Class, sh:NodeShape ;
		sh:rule [
			a sh:SPARQLRule ;
			sh:prefixes <Prefixes> ;
			sh:construct \"\"\"
				CONSTRUCT {	?employer :employee $this .	}
				WHERE {	$this :employer ?employer . }
				\"\"\" ;
		] ;
		sh:rule [
			a sh:SPARQLRule ;
			sh:prefixes <Prefixes> ;
			sh:construct \"\"\"
				CONSTRUCT {	$this :hasAncestor $x .	}
				WHERE {	$this :hasParent+ ?x . }
				\"\"\" ;
		] ;
		sh:rule [
			a sh:TripleRule ;
			sh:subject [sh:path :hasParent ] ;
			sh:predicate :hasChild ;
			sh:object sh:this 
    ] ;
		sh:rule [
			a sh:TripleRule ;
			sh:subject [sh:path [ sh:oneOrMorePath :hasParent ] ] ;
			sh:predicate :hasDescendant ;
			sh:object sh:this 
    ] .
""")


conforms, results_graph, results_text = shacl_validate_with_rules(dg,sg)
print(dg.serialize(format='turtle'))

# Task 1
The property :hasDescendant represents the transitive closure of property :hasChild.
Additional requirements (not checked by the tool, you have to check them yourself):
Solve this using a SPARQL rule.


In [ ]:
# Example
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Elizabeth>  :hasChild  <Charles> .

<Charles>  :hasChild  <William> , <Harry> .

<George>  :hasChild  <Elizabeth> .

<Harry>  :hasChild  <Archie> .
""")
print('-----------Before----------')
print(dg.serialize(format='turtle'))
print('-----------After----------')

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   <http://example.org/>

	<Prefixes> sh:declare 
  [ sh:prefix "rdf";
    sh:namespace "http://www.w3.org/1999/02/22-rdf-syntax-ns#"^^xsd:anyURI ] ,
  [ sh:prefix "rdfs";
    sh:namespace "http://www.w3.org/2000/01/rdf-schema#"^^xsd:anyURI ] ,
  [ sh:prefix "sh";
    sh:namespace "http://www.w3.org/ns/shacl#"^^xsd:anyURI ] ,
  [ sh:prefix "xsd";
    sh:namespace "http://www.w3.org/2001/XMLSchema#"^^xsd:anyURI ] ,
  [ sh:prefix "";
    sh:namespace "http://example.org/"^^xsd:anyURI ] .

    <PersonShape> a sh:NodeShape ;
  sh:targetSubjectsOf :hasChild ;
  sh:rule [
      a sh:SPARQLRule ;
      sh:prefixes <Prefixes> ;
      sh:construct \"\"\"
            CONSTRUCT { ?this :hasDescendant $x.}
            WHERE { $this :hasChild+ ?x .}
      \"\"\" ;
  ] .

""")

conforms, results_graph, results_text = shacl_validate_with_rules(dg,sg)
print(dg.serialize(format='turtle'))

# Task 2
The property :hasDescendant represents the transitive closure of property :hasChild.
Additional requirements (not checked by the tool, you have to check them yourself):
Solve this using a triple rule.


In [ ]:
# Example
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Elizabeth>  :hasChild  <Charles> .

<Charles>  :hasChild  <William> , <Harry> .

<George>  :hasChild  <Elizabeth> .

<Harry>  :hasChild  <Archie> .
""")
print('-----------Before----------')
print(dg.serialize(format='turtle'))
print('-----------After----------')

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   <http://example.org/>

	<Prefixes> sh:declare 
  [ sh:prefix "rdf";
    sh:namespace "http://www.w3.org/1999/02/22-rdf-syntax-ns#"^^xsd:anyURI ] ,
  [ sh:prefix "rdfs";
    sh:namespace "http://www.w3.org/2000/01/rdf-schema#"^^xsd:anyURI ] ,
  [ sh:prefix "sh";
    sh:namespace "http://www.w3.org/ns/shacl#"^^xsd:anyURI ] ,
  [ sh:prefix "xsd";
    sh:namespace "http://www.w3.org/2001/XMLSchema#"^^xsd:anyURI ] ,
  [ sh:prefix "";
    sh:namespace "http://example.org/"^^xsd:anyURI ] .

<PersonShape> a sh:NodeShape ;
  sh:targetSubjectsOf :hasChild ;
  sh:rule [
      a sh:TripleRule ;
      sh:subject sh:this ;
      sh:predicate :hasDescendant ;
      sh:object [sh:path [sh:zeroOrMorePath :hasChild]] ;
  ] .


""")


conforms, results_graph, results_text = shacl_validate_with_rules(dg,sg)
print(dg.serialize(format='turtle'))

# Task 3
The tax rate of a product category is propagated to products.
Additional requirements (not checked by the tool, you have to check them yourself):
Solve this using a triple rule.


In [ ]:
# Example
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<HP4>   rdf:type          <Product> ;
        :price            12 ;
        :productCategory  <Book> .

<Porsche911>  rdf:type    <Product> ;
        :price            121000 ;
        :productCategory  <Car> .

<VolvoV50>  rdf:type      <Product> ;
        :price            27000 ;
        :productCategory  <Car> .

<Car>   :taxRate  0.2 .

<Book>  :taxRate  0.1 .
""")
print('-----------Before----------')
print(dg.serialize(format='turtle'))
print('-----------After----------')

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   <http://example.org/>

	<Prefixes> sh:declare 
  [ sh:prefix "rdf";
    sh:namespace "http://www.w3.org/1999/02/22-rdf-syntax-ns#"^^xsd:anyURI ] ,
  [ sh:prefix "rdfs";
    sh:namespace "http://www.w3.org/2000/01/rdf-schema#"^^xsd:anyURI ] ,
  [ sh:prefix "sh";
    sh:namespace "http://www.w3.org/ns/shacl#"^^xsd:anyURI ] ,
  [ sh:prefix "xsd";
    sh:namespace "http://www.w3.org/2001/XMLSchema#"^^xsd:anyURI ] ,
  [ sh:prefix "";
    sh:namespace "http://example.org/"^^xsd:anyURI ] .

<TaxShape> a sh:NodeShape ;
  sh:targetClass <Product> ;
  sh:rule [
      a sh:TripleRule ;
      sh:subject sh:this;
      sh:predicate :taxRate ;
      sh:object [
        sh:path (:productCategory :taxRate)
      ]
  ] .


""")


conforms, results_graph, results_text = shacl_validate_with_rules(dg,sg)
print(dg.serialize(format='turtle'))

# Task 4
Properties :hasMother and :hasFather are derived from properties :hasChild and classes Man and Woman.
Additional requirements (not checked by the tool, you have to check them yourself):
Derive :hasFather statements using a triple rule defined as part of a node shape that has Man as target class.
Derive :hasMother statements using a triple rule defined as part of a node shape that has Woman as target class.

In [ ]:
# Example
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Diana>  rdf:type  <Woman> ;
        :hasChild  <William> , <Harry> .

<William>  rdf:type  <Man> .

<Harry>  rdf:type  <Man> ;
        :hasChild  <Archie> .

<Man>   rdfs:subClassOf  <Person> .

<Charles>  rdf:type  <Man> ;
        :hasChild  <William> , <Harry> .

<Archie>  rdf:type  <Person> .

<Elizabeth>  rdf:type  <Woman> ;
        :hasChild  <Charles> .

<Woman>  rdfs:subClassOf  <Person> .
""")
print('-----------Before----------')
print(dg.serialize(format='turtle'))
print('-----------After----------')

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   <http://example.org/>

	<Prefixes> sh:declare 
  [ sh:prefix "rdf";
    sh:namespace "http://www.w3.org/1999/02/22-rdf-syntax-ns#"^^xsd:anyURI ] ,
  [ sh:prefix "rdfs";
    sh:namespace "http://www.w3.org/2000/01/rdf-schema#"^^xsd:anyURI ] ,
  [ sh:prefix "sh";
    sh:namespace "http://www.w3.org/ns/shacl#"^^xsd:anyURI ] ,
  [ sh:prefix "xsd";
    sh:namespace "http://www.w3.org/2001/XMLSchema#"^^xsd:anyURI ] ,
  [ sh:prefix "";
    sh:namespace "http://example.org/"^^xsd:anyURI ] .

<ManShape> a sh:NodeShape ;
  sh:targetClass <Man> ;
  sh:rule [
      a sh:TripleRule ;
      sh:subject [sh:path :hasChild ] ;
      sh:predicate :hasFather ;
      sh:object sh:this
  ].

<WomanShape> a sh:NodeShape ;
  sh:targetClass <Woman> ;
  sh:rule [
      a sh:TripleRule ;
      sh:subject [sh:path :hasChild ] ;
      sh:predicate :hasMother ;
      sh:object sh:this
  ].

""")


conforms, results_graph, results_text = shacl_validate_with_rules(dg,sg)
print(dg.serialize(format='turtle'))

# Task 5
Properties :hasMother and :hasFather are derived from properties :hasChild and classes Man and Woman.
Additional requirements (not checked by the tool, you have to check them yourself):
Derive :hasFather statements as well as :hasMother statements using triple rules defined as part of a node shape that has Person as target class.

In [ ]:
# Example
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Diana>  rdf:type  <Woman> ;
        :hasChild  <William> , <Harry> .

<William>  rdf:type  <Man> .

<Harry>  rdf:type  <Man> ;
        :hasChild  <Archie> .

<Man>   rdfs:subClassOf  <Person> .

<Charles>  rdf:type  <Man> ;
        :hasChild  <William> , <Harry> .

<Archie>  rdf:type  <Person> .

<Elizabeth>  rdf:type  <Woman> ;
        :hasChild  <Charles> .

<Woman>  rdfs:subClassOf  <Person> .
""")
print('-----------Before----------')
print(dg.serialize(format='turtle'))
print('-----------After----------')

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   <http://example.org/>

	<Prefixes> sh:declare 
  [ sh:prefix "rdf";
    sh:namespace "http://www.w3.org/1999/02/22-rdf-syntax-ns#"^^xsd:anyURI ] ,
  [ sh:prefix "rdfs";
    sh:namespace "http://www.w3.org/2000/01/rdf-schema#"^^xsd:anyURI ] ,
  [ sh:prefix "sh";
    sh:namespace "http://www.w3.org/ns/shacl#"^^xsd:anyURI ] ,
  [ sh:prefix "xsd";
    sh:namespace "http://www.w3.org/2001/XMLSchema#"^^xsd:anyURI ] ,
  [ sh:prefix "";
    sh:namespace "http://example.org/"^^xsd:anyURI ] .

""")

# RuleLoadError: A SHACL Rule Condition must be an existing well-formed SHACL Shape.
# For reference, see https://www.w3.org/TR/shacl-af/#condition
semAi = """
<PersonShape> a sh:NodeShape ;
  sh:targetClass <Person> ;
  sh:rule [
      a sh:TripleRule ;
      sh:subject [sh:path :hasChild ] ;
      sh:predicate :hasFather ;
      sh:object sh:this;
      sh:condition [ sh:class <Man>; ]
  ] ;
  sh:rule [
      a sh:TripleRule ;
      sh:subject [sh:path :hasChild ] ;
      sh:predicate :hasMother ;
      sh:object sh:this;
      sh:condition [ sh:class <Woman>; ]
  ] .

"""


conforms, results_graph, results_text = shacl_validate_with_rules(dg,sg)
print(dg.serialize(format='turtle'))

# Task 6
The average weight of a species should be aggregated from the individual animals.
(This can be solved using a SPARQL rule

In [ ]:
# Example
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Collie>  rdfs:subClassOf  <Dog> .

<Garfield>  rdf:type  <Cat> ;
        :weight   12 .

<Bello>  rdf:type  <Dog> ;
        :weight   27 .

<Cat>   rdf:type  <Species> .

<Lassie>  rdf:type  <Collie> ;
        :weight   31 .

<Tom>   rdf:type  <Cat> ;
        :weight   6 .

<Dog>   rdf:type  <Species> .
""")
print('-----------Before----------')
print(dg.serialize(format='turtle'))
print('-----------After----------')

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   <http://example.org/>

	<Prefixes> sh:declare 
  [ sh:prefix "rdf";
    sh:namespace "http://www.w3.org/1999/02/22-rdf-syntax-ns#"^^xsd:anyURI ] ,
  [ sh:prefix "rdfs";
    sh:namespace "http://www.w3.org/2000/01/rdf-schema#"^^xsd:anyURI ] ,
  [ sh:prefix "sh";
    sh:namespace "http://www.w3.org/ns/shacl#"^^xsd:anyURI ] ,
  [ sh:prefix "xsd";
    sh:namespace "http://www.w3.org/2001/XMLSchema#"^^xsd:anyURI ] ,
  [ sh:prefix "";
    sh:namespace "http://example.org/"^^xsd:anyURI ] .

<WeightShape> a sh:NodeShape ;
  sh:targetClass <Species> ;
  sh:rule [
      a sh:SPARQLRule ;
      sh:prefixes <Prefixes> ;
      sh:construct \"\"\"
            CONSTRUCT { $this :avgWeight ?avg.}
            WHERE { 
               SELECT $this (AVG(?weight) AS ?avg)
               WHERE { 
                   ?sub rdf:type ?super;
                      :weight ?weight.
                   ?super rdfs:subClassOf* $this . 
               }
               GROUP BY $this
            }
      \"\"\" ;
  ] .
""")


conforms, results_graph, results_text = shacl_validate_with_rules(dg,sg)
print(dg.serialize(format='turtle'))

Anmerkung für Präsentation:
* "zero or more" property path operator: *
* "one or more" property path operator: +
* "zero or one" property path operator: ?


# Task 7
The area of a rectangle is the product of its length and its width.
Additional requirements (not checked by the tool, you have to check them yourself):
Solve this using a SPARQL rule



In [ ]:
# Example
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Peter>  rdf:type  <Person> ;
        :length   1.72 ;
        :width    .9 .

<Rect2>  rdf:type  <Rectangle> ;
        :length   5.5 ;
        :width    4 .

<Rect1>  rdf:type  <Rectangle> ;
        :length   10 ;
        :width    6 .
""")
print('-----------Before----------')
print(dg.serialize(format='turtle'))
print('-----------After----------')

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   <http://example.org/>

	<Prefixes> sh:declare 
  [ sh:prefix "rdf";
    sh:namespace "http://www.w3.org/1999/02/22-rdf-syntax-ns#"^^xsd:anyURI ] ,
  [ sh:prefix "rdfs";
    sh:namespace "http://www.w3.org/2000/01/rdf-schema#"^^xsd:anyURI ] ,
  [ sh:prefix "sh";
    sh:namespace "http://www.w3.org/ns/shacl#"^^xsd:anyURI ] ,
  [ sh:prefix "xsd";
    sh:namespace "http://www.w3.org/2001/XMLSchema#"^^xsd:anyURI ] ,
  [ sh:prefix "";
    sh:namespace "http://example.org/"^^xsd:anyURI ] .

<RecShape> a sh:NodeShape ;
  sh:targetClass <Rectangle> ;
  sh:rule [
      a sh:SPARQLRule ;
      sh:prefixes <Prefixes> ;
      sh:construct \"\"\"
            CONSTRUCT { $this :area ?area.}
            WHERE { 
              $this :length ?l;
                    :width ?w . 
              BIND (?w * ?l AS ?area) 
            }
      \"\"\" ;
  ] .
""")


conforms, results_graph, results_text = shacl_validate_with_rules(dg,sg)
print(dg.serialize(format='turtle'))

# Task 8
The area of a rectangle is the product of its length and its width.
Additional requirements (not checked by the tool, you have to check them yourself):
Solve this using a SHACL function :multiply together with a triple rule.


In [ ]:
# Example
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Peter>  rdf:type  <Person> ;
        :length   1.72 ;
        :width    .9 .

<Rect2>  rdf:type  <Rectangle> ;
        :length   5.5 ;
        :width    4 .

<Rect1>  rdf:type  <Rectangle> ;
        :length   10 ;
        :width    6 .
""")
print('-----------Before----------')
print(dg.serialize(format='turtle'))
print('-----------After----------')

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   <http://example.org/>

	<Prefixes> sh:declare 
  [ sh:prefix "rdf";
    sh:namespace "http://www.w3.org/1999/02/22-rdf-syntax-ns#"^^xsd:anyURI ] ,
  [ sh:prefix "rdfs";
    sh:namespace "http://www.w3.org/2000/01/rdf-schema#"^^xsd:anyURI ] ,
  [ sh:prefix "sh";
    sh:namespace "http://www.w3.org/ns/shacl#"^^xsd:anyURI ] ,
  [ sh:prefix "xsd";
    sh:namespace "http://www.w3.org/2001/XMLSchema#"^^xsd:anyURI ] ,
  [ sh:prefix "";
    sh:namespace "http://example.org/"^^xsd:anyURI ] .

:multiply
	a sh:SPARQLFunction ;
	rdfs:comment "Multiplies its two arguments $op1 and $op2." ;
	sh:parameter [
		sh:path :op1 ;
		sh:datatype :integer ;
		sh:description "The first operand" ;
	] ;
	sh:parameter [
		sh:path :op2 ;
		sh:datatype xsd:integer ;
		sh:description "The second operand" ;
	] ;
	sh:returnType xsd:integer ;
	sh:select \"\"\"
		SELECT ($op1 * $op2 AS ?result)
		WHERE {
		}
		\"\"\" .

<RecTRShape> a sh:NodeShape ;
  sh:targetClass <Rectangle> ;
  sh:rule [
      a sh:TripleRule ;
      sh:subject sh:this ;
      sh:predicate :area ;
      sh:object [
          :multiply([sh:path :width] [sh:path :length])
      ]
  ] .
""")

# ReportableRuntimeError: The SHACLFunction http://example.org/multiply was not defined in this SHACL Shapes file.
semAI = """
<RecTRShape> a sh:NodeShape ;
  sh:targetClass <Rectangle> ;
  sh:rule [
      a sh:TripleRule ;
      sh:subject sh:this ;
      sh:predicate :area ;
      sh:object [
          :multiply([sh:path :width] [sh:path :length])
      ]
  ] .
"""


conforms, results_graph, results_text = shacl_validate_with_rules(dg,sg)
print(dg.serialize(format='turtle'))

# Task 9
The body mass index (:bmi) of a person is its weight divided by its height squared.
The bmi should be rounded to 2 decimal places.
Additional requirements (not checked by the tool, you have to check them yourself):
Solve this using a SPARQL rule.


In [ ]:
# Example
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Bello>  rdf:type  <Dog> ;
        :height   0.77 ;
        :weight   27 .

<Peter>  rdf:type  <Person> ;
        :height   1.78 ;
        :weight   86 .

<Jane>  rdf:type  <Person> ;
        :height   1.72 ;
        :weight   72 .
""")
print('-----------Before----------')
print(dg.serialize(format='turtle'))
print('-----------After----------')

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   <http://example.org/>

	<Prefixes> sh:declare 
  [ sh:prefix "rdf";
    sh:namespace "http://www.w3.org/1999/02/22-rdf-syntax-ns#"^^xsd:anyURI ] ,
  [ sh:prefix "rdfs";
    sh:namespace "http://www.w3.org/2000/01/rdf-schema#"^^xsd:anyURI ] ,
  [ sh:prefix "sh";
    sh:namespace "http://www.w3.org/ns/shacl#"^^xsd:anyURI ] ,
  [ sh:prefix "xsd";
    sh:namespace "http://www.w3.org/2001/XMLSchema#"^^xsd:anyURI ] ,
  [ sh:prefix "";
    sh:namespace "http://example.org/"^^xsd:anyURI ] .

<PersonShape> a sh:NodeShape ;
  sh:targetClass <Person> ;
  sh:rule [
      a sh:SPARQLRule ;
      sh:prefixes <Prefixes> ;
      sh:construct \"\"\"
            CONSTRUCT { $this :bmi ?bmi.}
            WHERE { 
              $this :height ?h ;
                    :weight ?w .
              BIND((ROUND((?w / (?h * ?h))*100))/100 AS ?bmi) .
            }
      \"\"\" ;
  ] .
""")


conforms, results_graph, results_text = shacl_validate_with_rules(dg,sg)
print(dg.serialize(format='turtle'))

# Task 10
The body mass index (:bmi) of a person is its weight divided by its height squared.
The bmi should be rounded to 2 decimal places.
Additional requirements (not checked by the tool, you have to check them yourself):
Solve this using SHACL functions (one for multiply and one for division rounded to two decimal places) and a triple rule.


In [ ]:
# Example
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Bello>  rdf:type  <Dog> ;
        :height   0.77 ;
        :weight   27 .

<Peter>  rdf:type  <Person> ;
        :height   1.78 ;
        :weight   86 .

<Jane>  rdf:type  <Person> ;
        :height   1.72 ;
        :weight   72 .
""")
print('-----------Before----------')
print(dg.serialize(format='turtle'))
print('-----------After----------')

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   <http://example.org/>

	<Prefixes> sh:declare 
  [ sh:prefix "rdf";
    sh:namespace "http://www.w3.org/1999/02/22-rdf-syntax-ns#"^^xsd:anyURI ] ,
  [ sh:prefix "rdfs";
    sh:namespace "http://www.w3.org/2000/01/rdf-schema#"^^xsd:anyURI ] ,
  [ sh:prefix "sh";
    sh:namespace "http://www.w3.org/ns/shacl#"^^xsd:anyURI ] ,
  [ sh:prefix "xsd";
    sh:namespace "http://www.w3.org/2001/XMLSchema#"^^xsd:anyURI ] ,
  [ sh:prefix "";
    sh:namespace "http://example.org/"^^xsd:anyURI ] .

:multiply
	a sh:SPARQLFunction ;
	rdfs:comment "Multiplies its two arguments $op1 and $op2." ;
	sh:parameter [
		sh:path :op1 ;
		sh:datatype :integer ;
		sh:description "The first operand" ;
	] ;
	sh:parameter [
		sh:path :op2 ;
		sh:datatype xsd:integer ;
		sh:description "The second operand" ;
	] ;
	sh:returnType xsd:integer ;
	sh:select \"\"\"
		SELECT ($op1 * $op2 AS ?result)
		WHERE {
		}
		\"\"\" .

:divide
	a sh:SPARQLFunction ;
	rdfs:comment "Divide its two arguments $op1 and $op2." ;
	sh:parameter [
		sh:path :op1 ;
		sh:datatype :integer ;
		sh:description "The first operand" ;
	] ;
	sh:parameter [
		sh:path :op2 ;
		sh:datatype xsd:integer ;
		sh:description "The second operand" ;
	] ;
	sh:returnType xsd:integer ;
	sh:select \"\"\"
		SELECT ($op1 / $op2 AS ?result)
		WHERE {
		}
		\"\"\" .


<PersonShape> a sh:NodeShape ;
  sh:targetClass <Person> ;
  sh:rule [
      a sh:TripleRule ;
      sh:subject sh:this ;
      sh:predicate :bmi ;
      sh:object [
         :divide (
           [sh:path :weight][ 
           :multiply (
               [sh:path :height][sh:path :height]
            )]
          )
      ]

  ] .
""")

# ReportableRuntimeError: The SHACLFunction http://example.org/divide was not defined in this SHACL Shapes file.

conforms, results_graph, results_text = shacl_validate_with_rules(dg,sg)
print(dg.serialize(format='turtle'))